In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from random import seed
from random import randrange

In [2]:
# Lectura de datos
data = pd.read_csv('iris.data',header=None)
data.columns = ['Sepal.length', 'Sepal.width', 'Petal.length', 'Petal.width', 'Species']

In [8]:
# Función de clasificación
def classificador(x, y, theta, theta_0, tipo):
    z = y*(np.dot(x, theta) + theta_0)
    
    if z < 1.0:
        z = -1
    else:
        z = 0
    
    if tipo == 1:
        z = z*y*x
    else:
        z = z*y
    
    return z

# Función de pérdida
def loess(X, Y, theta, theta_0, etha, lamb):
    nr, nc = X.shape
    suma_theta = 0
    suma_theta_0 = 0
    
    for i in range(nr):
        specie = Y.loc[i][0]
        # Positivo: Setosa, Negativo: el resto
        if specie == 'Iris-setosa':
            y = 1
        else:
            y = -1
        suma_theta = suma_theta + classificador(X.loc[i], y, theta, theta_0, tipo = 1)
        suma_theta_0 = suma_theta_0 + classificador(X.loc[i], y, theta, theta_0, tipo = 2)
        
    theta = theta - etha*( (1/nr)*suma_theta + lamb*theta)
    theta_0 = theta_0 - etha*(1/nr)*suma_theta_0
    return theta, theta_0

# Jacobiano
def jacob(X, Y, theta, theta_0, lamb):
    nr, nc = X.shape
    suma_jac = 0
    
    for i in range(nr):
        specie = Y.loc[i][0]
        # Positivo: Setosa, Negativo: el resto
        if specie == 'Iris-setosa':
            y = 1
        else:
            y = -1
        v = y*(np.dot(X.loc[i], theta) + theta_0)
        if v < 1.0:
            L = 1-v
        else:
            L = 0
        suma_jac = suma_jac + L
    
    jac = (1/nr)*suma_jac + (lamb/2)*np.dot(theta, theta)
    return jac

# Método de descenso por gradiente
def descenso_gradiente(X, Y, theta, theta_0, etha, lamb, MAX = 100, tol = 1e-4):
    t = 0
    error = 10
    while error >= tol and t < MAX:
        t = t + 1
        theta_act, theta_0_act = loess(X, Y, theta, theta_0, etha, lamb)
        error = abs(jacob(X, Y, theta_act, theta_0_act, lamb) - jacob(X, Y, theta, theta_0, lamb))
        #print('Error: ', error)
        #print('Iteracion: ', t)
        theta = theta_act
        theta_0 = theta_0_act
    return theta_act, theta_0_act

# Cross validation
def cv_split(data, k):
    """esta funcion tiene de imput un dataset y un numero de fragmentos y de output las particiones"""
    df_split = list()
    df_copy = data
    tamaño_particion = int(len(data) / k)
    for i in range(k):
        particion = list()
        while len(particion) < tamaño_particion:
            index = randrange(len(df_copy))
            particion.append(df_copy.pop(index))
        df_split.append(particion)
    return df_split
############todavia no sirve, si lo puedes implementar con un df estaria chido##########

In [11]:
X = data[['Sepal.length', 'Sepal.width', 'Petal.length' , 'Petal.width']]
Y = data[['Species']]
theta = np.array([0.0, 0.0, 0.0, 0.0])
theta_0 = 1.0
etha = 0.1
lamb = 0.1

In [12]:
theta_f, theta_0_f = descenso_gradiente(X, Y, theta, theta_0, etha, lamb)

In [14]:
jacob(X, Y, theta_f, theta_0_f, lamb)

0.051791675594372616